In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import os
from datetime import datetime

from ff_app.execution import run_data_pull, update_google_sheet
from ff_app import data_prep

04:05:15 INFO:NumExpr defaulting to 8 threads.


In [14]:
week = 3

short_df = run_data_pull(week=week)
# short_df, all_df = run_data_pull(week=week, return_all_games=True)

print(short_df.shape)

08:55:12 DEBUG:Starting new HTTPS connection (1): api.the-odds-api.com:443
08:55:13 DEBUG:https://api.the-odds-api.com:443 "GET /v4/sports/americanfootball_ncaaf/odds?markets=spreads%2Ctotals&regions=us&oddsFormat=american&dateFormat=iso&api_key=ce31cd809f1458374e07be4035c3d210 HTTP/1.1" 200 15300
08:55:13 INFO:Pulled 54 total games for week 3 in 2023
08:55:13 INFO:Picks sheet data has shape (54, 10)
08:55:13 INFO:Saving game data to disk at '/Users/hoke/dev/football/game_lists/2023/week3.csv'


(54, 10)


In [15]:
short_df.head()

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Implied Score
0,3,09/14 07:30 PM (Thu),N,Memphis Tigers,Navy Midshipmen,Memphis Tigers,Navy Midshipmen,-14.5,47.0,30 - 16
1,3,09/15 07:00 PM (Fri),N,UTSA Roadrunners,Army Black Knights,UTSA Roadrunners,Army Black Knights,-8.5,45.0,26 - 18
2,3,09/15 07:00 PM (Fri),N,Maryland Terrapins,Virginia Cavaliers,Maryland Terrapins,Virginia Cavaliers,-14.5,49.0,31 - 17
3,3,09/15 08:00 PM (Fri),N,Air Force Falcons,Utah State Aggies,Air Force Falcons,Utah State Aggies,-10.0,46.5,28 - 18
4,3,09/16 12:00 PM (Sat),N,Mississippi State Bulldogs,LSU Tigers,LSU Tigers,Mississippi State Bulldogs,-9.5,54.5,32 - 22


In [17]:
check_team = 'Clemson'

short_df[(short_df['Favorite'].str.contains(check_team)) | (short_df['Underdog'].str.contains(check_team))]

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Implied Score
45,3,09/16 08:00 PM (Sat),N,Clemson Tigers,Florida Atlantic Owls,Clemson Tigers,Florida Atlantic Owls,-24.0,52.0,38 - 14


In [18]:
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Implied Score


In [20]:
team_list = [
    'Minnesota',
    'Tennessee',
    'Duke Blue Devils',
    'Bowling Green',
    'West Virginia'
]

all_team_list = short_df['Favorite'].tolist() + short_df['Underdog'].tolist()
mandatory_list = []
for team in team_list:
    for t in all_team_list:
        if team in t:
            mandatory_list.append(t)

mandatory_picks = short_df[(short_df['Favorite'].isin(mandatory_list)) |
                           (short_df['Underdog'].isin(mandatory_list))]
mandatory_picks

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Implied Score
17,3,09/16 03:30 PM (Sat),N,North Carolina Tar Heels,Minnesota Golden Gophers,North Carolina Tar Heels,Minnesota Golden Gophers,-7.5,50.0,28 - 21
19,3,09/16 03:30 PM (Sat),N,Duke Blue Devils,Northwestern Wildcats,Duke Blue Devils,Northwestern Wildcats,-18.5,48.5,33 - 15
31,3,09/16 07:00 PM (Sat),N,Florida Gators,Tennessee Volunteers,Tennessee Volunteers,Florida Gators,-6.5,59.0,32 - 26
40,3,09/16 07:30 PM (Sat),N,West Virginia Mountaineers,Pittsburgh Panthers,Pittsburgh Panthers,West Virginia Mountaineers,-1.0,50.0,25 - 24
44,3,09/16 07:30 PM (Sat),N,Michigan Wolverines,Bowling Green Falcons,Michigan Wolverines,Bowling Green Falcons,-40.5,53.5,47 - 6


In [21]:
short_df.loc[mandatory_picks.index.tolist(), 'Mandatory'] = 'Y'
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Implied Score
17,3,09/16 03:30 PM (Sat),Y,North Carolina Tar Heels,Minnesota Golden Gophers,North Carolina Tar Heels,Minnesota Golden Gophers,-7.5,50.0,28 - 21
19,3,09/16 03:30 PM (Sat),Y,Duke Blue Devils,Northwestern Wildcats,Duke Blue Devils,Northwestern Wildcats,-18.5,48.5,33 - 15
31,3,09/16 07:00 PM (Sat),Y,Florida Gators,Tennessee Volunteers,Tennessee Volunteers,Florida Gators,-6.5,59.0,32 - 26
40,3,09/16 07:30 PM (Sat),Y,West Virginia Mountaineers,Pittsburgh Panthers,Pittsburgh Panthers,West Virginia Mountaineers,-1.0,50.0,25 - 24
44,3,09/16 07:30 PM (Sat),Y,Michigan Wolverines,Bowling Green Falcons,Michigan Wolverines,Bowling Green Falcons,-40.5,53.5,47 - 6


In [22]:
update_google_sheet(short_df)

08:56:09 INFO:Using credentials directory '/Users/hoke/dev/football/credentials'
08:56:09 INFO:Obtained credentails from existing 'token.json' file
08:56:09 INFO:Obtaining a token using credentials stored in 'credentials.json
08:56:09 DEBUG:Generated new state Baee8i3V8bule2YQcCeK334I3mgprA.


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=442514865556-pp8un74st0drc1kdpurs4j8bmfav10gs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64334%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=Baee8i3V8bule2YQcCeK334I3mgprA&access_type=offline


08:56:20 INFO:"GET /?state=Baee8i3V8bule2YQcCeK334I3mgprA&code=4/0Adeu5BVo0mYUrstPwXMDSWIYE1iPCmD79cuDzs9CYRgkA8LG6kyNz5I7gEEPORw-8MubdA&scope=https://www.googleapis.com/auth/spreadsheets HTTP/1.1" 200 65
08:56:20 DEBUG:Encoding `client_id` "442514865556-pp8un74st0drc1kdpurs4j8bmfav10gs.apps.googleusercontent.com" with `client_secret` as Basic auth credentials.
08:56:20 DEBUG:Requesting url https://oauth2.googleapis.com/token using method POST.
08:56:20 DEBUG:Supplying headers {'Accept': 'application/json', 'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'} and data {'grant_type': 'authorization_code', 'code': '4/0Adeu5BVo0mYUrstPwXMDSWIYE1iPCmD79cuDzs9CYRgkA8LG6kyNz5I7gEEPORw-8MubdA', 'redirect_uri': 'http://localhost:64334/'}
08:56:20 DEBUG:Passing through key word arguments {'timeout': None, 'auth': <requests.auth.HTTPBasicAuth object at 0x7f82605b34f0>, 'verify': True, 'proxies': None}.
08:56:20 DEBUG:Starting new HTTPS connection (1): oauth2.googleapis.com:443
08:5